In [45]:
# imports
import panel as pn
pn.extension('plotly')
import pandas as pd
import plotly.express as px
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()
map_box_api = os.getenv("mapbox")
from hvplot import hvPlot
import holoviews as hv
hv.extension('bokeh')

# Read the 2020 & 2021 data into a Pandas DataFrame
file_path_2020 = Path("2020.csv")
file_path_2021 = Path("2021.csv")
file_path_3 = Path("COVID-19_by_Province.csv")
Mobility_2020_data = pd.read_csv(file_path_2020)
Mobility_2021_data = pd.read_csv(file_path_2021)
Hospitalisation_data = pd.read_csv(file_path_3)
Mobility_data_actual = pd.concat([Mobility_2020_data,Mobility_2021_data])

# Drop columns and save the output in a csv which will be the final database to be used for analysis
Mobility_data_edit_1 = Mobility_data_actual.drop(columns = ['country_region_code','country_region','sub_region_2','metro_area','iso_3166_2_code','census_fips_code','place_id'])
Mobility_data_edit_2 = Mobility_data_edit_1.dropna()
Mobility_data_edit_3 = Mobility_data_edit_2.set_index('date')
Mobility_data_edit_4 = Mobility_data_edit_2.rename(columns = {'sub_region_1':'Province', 
                                                              'retail_and_recreation_percent_change_from_baseline':'Retail & Recreation',
                                                             'grocery_and_pharmacy_percent_change_from_baseline':'Grocery & Pharmacy',
                                                              'parks_percent_change_from_baseline':'Parks',
                                                              'transit_stations_percent_change_from_baseline':'Transit Stations',
                                                              'workplaces_percent_change_from_baseline':'Workplaces',
                                                              'residential_percent_change_from_baseline':'Residential'
                                                             })
Mobility_data_edit_5 = Mobility_data_edit_4.groupby(["date","Province"]).mean()                         
Mobility_data_edit_5.to_csv("Mobility_Data_Canada_2020_21.csv")
Mobility_data_edit_6 = Mobility_data_edit_5.reset_index()

Hospitalisation_data_1 = Hospitalisation_data.rename(columns = {'GEO':'Province'})
Hospitalisation_data_2 = Hospitalisation_data_1.set_index('Date')


#Combine datapoints of hospitalisation and mobility data
Mob_hosp_data = pd.merge(Mobility_data_edit_6, Hospitalisation_data_2,how = 'left', 
                         left_on=['date','Province'],
                         right_on=['Date','Province']
                        )
Mob_hosp_data.dropna()
Mob_hosp_data['date']=pd.to_datetime(Mob_hosp_data['date'])
Mob_hosp_data = Mob_hosp_data.set_index('date')

# Visualisation of data province-wise for all the places visited
Mob_hosp_data.hvplot.line(x="date",y="Retail & Recreation",groupby="Province",rot = 90)

Mob_hosp_data.hvplot.line(x="date",y="Grocery & Pharmacy",groupby="Province",rot = 90)

Mob_hosp_data.hvplot.line(x="date",y="Parks",groupby="Province",rot = 90)

Mob_hosp_data.hvplot.line(x="date",y="Transit Stations",groupby="Province",rot = 90)

Mob_hosp_data.hvplot.line(x="date",y="Workplaces",groupby="Province",rot = 90)

Mob_hosp_data.hvplot.line(x="date",y="Residential",groupby="Province",rot = 90)

#Hospitalisation data based on mobility to places mentioned
Mob_hosp_data_Canada = Mob_hosp_data.groupby("date").mean()
Mob_hosp_data_Canada_1 = Mob_hosp_data_Canada.dropna()
plot = hvPlot(Mob_hosp_data_Canada_1)
plot(y=['Retail & Recreation', 
        'Grocery & Pharmacy', 
        'Parks', 
        'Transit Stations', 
        'Workplaces', 
        'Residential',
       'Hospitalizations'], width = 900, legend = 'top' )

